In [1]:
# Import the libraries
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Declare the credentials
cid = 'a1e7da2d0baf48388ecafad2445703f1'
secret = 'confidential'
redirect_uri='http://localhost:7777/callback?code=AQC3CKowTatcXrBbIiln3rYCGm0u86rgmPEaV4u-UwlN7Cx-7FYAyU3m6T6GVyD4yjRE4bWb1P6qRefp4hYaktQ7Hhsq09g_x22cxzD8XwmECFOB9Q4c9T-9diC_72na1s1FP0gnLMId71rh9yf8y31S6JjPjNieAZWZ1xOkElRaFaIzcp-w4Lk-SCyi8D-qBA'
username = 'm6hy7k7xm1k8gte1pk9r2lmbq'

In [3]:
# Authorization flow
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri='http://localhost:7777/callback?code=AQC3CKowTatcXrBbIiln3rYCGm0u86rgmPEaV4u-UwlN7Cx-7FYAyU3m6T6GVyD4yjRE4bWb1P6qRefp4hYaktQ7Hhsq09g_x22cxzD8XwmECFOB9Q4c9T-9diC_72na1s1FP0gnLMId71rh9yf8y31S6JjPjNieAZWZ1xOkElRaFaIzcp-w4Lk-SCyi8D-qBA')

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

### Extract User Playlist

In [4]:
# Fetch the user's playlists
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlists = sp.user_playlists(username)
for playlist in playlists['items']:
    print('Name: {}, Playlist ID: {}, No Tracks: {}'.
          format(playlist['name'], playlist['id'], playlist['tracks']['total']))

Name: Spotify's Most Played All-Time [Updated Weekly] | Most Streamed | Top Played | 500Mil+, Playlist ID: 2YRe7HRKNRvXdJBp9nXFza, No Tracks: 791
Name: My playlist, Playlist ID: 3XkRHqmlECMtmVSWfaoHKV, No Tracks: 8


In [5]:
def fetch_playlists(sp, username):
    """
    Returns the user's playlists.
    """
        
    id = []
    name = []
    num_tracks = []
    
    # Make the API request
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        id.append(playlist['id'])
        name.append(playlist['name'])
        num_tracks.append(playlist['tracks']['total'])

    # Create the final df   
    df_playlists = pd.DataFrame({"id":id, "name": name, "#tracks": num_tracks})
    return df_playlists

In [6]:
fetch_playlists(sp,username)

,id,name,#tracks
0,2YRe7HRKNRvXdJBp9nXFza,Spotify's Most Played All-Time [Updated Weekly...,791
1,3XkRHqmlECMtmVSWfaoHKV,My playlist,8


In [7]:
def fetch_playlist_tracks(sp, username, playlist_id):
    """
    Returns the tracks for the given playlist.
    """
        
    offset = 0
    tracks = []
    
    # Make the API request
    while True:
        content = sp.user_playlist_tracks(username, playlist_id, fields=None, limit=100, offset=offset, market=None)
        tracks += content['items']
        
        if content['next'] is not None:
            offset += 100
        else:
            break
    
    track_id = []
    track_name = []
    
    for track in tracks:
        track_id.append(track['track']['id'])
        track_name.append(track['track']['name'])
    
    # Create the final df
    df_playlists_tracks = pd.DataFrame({"track_id":track_id, "track_name": track_name})
    return df_playlists_tracks

In [8]:
fetch_playlist_tracks(sp, username, '2YRe7HRKNRvXdJBp9nXFza')

,track_id,track_name
0,7qiZfU4dY1lWllzX7mPBI3,Shape of You
1,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights
2,1rgnBhdG2JDFTbYkYRZAku,Dance Monkey
3,0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage)
4,7qEHsqek33rTcFNT9PFqLf,Someone You Loved
...,...,...
786,4EWCNWgDS8707fNSZ1oaA5,Heartless
787,67iAlVNDDdddxqSD2EZhFs,I'm Gonna Be (500 Miles)
788,1dGr1c8CrMLDpV6mPbImSI,Lover
789,2iUmqdfGZcHIhS3b9E9EWq,Everybody Talks


### Extract the Tracks' Audio Features

In [9]:
def fetch_audio_features(sp, username, playlist_id):
    playlist = fetch_playlist_tracks(sp, username, playlist_id)
    index = 0
    audio_features = []
    
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    features_list = []
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['energy'], features['tempo'],
                              features['loudness'], features['valence'],
                              features['speechiness'], features['instrumentalness'],
                              features['liveness'], features['acousticness']])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'energy',
                                                             'tempo', 'loudness', 'valence',
                                                             'speechiness', 'instrumentalness',
                                                             'liveness', 'acousticness'])
    
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_id', inplace=True, drop=True)
        
    return df_playlist_audio_features

In [39]:
df = fetch_audio_features(sp, username, '2YRe7HRKNRvXdJBp9nXFza')
display(df.shape)
df.head(50)

(791, 10)

,track_name,danceability,energy,tempo,loudness,valence,speechiness,instrumentalness,liveness,acousticness
track_id,,,,,,,,,,
7qiZfU4dY1lWllzX7mPBI3,Shape of You,0.825,0.652,95.977,-3.183,0.931,0.0802,0.000000,0.0931,0.581000
0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,0.514,0.730,171.005,-5.934,0.334,0.0598,0.000095,0.0897,0.001460
1rgnBhdG2JDFTbYkYRZAku,Dance Monkey,0.826,0.593,98.083,-6.401,0.541,0.0976,0.000161,0.1700,0.688000
0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage),0.585,0.520,159.801,-6.136,0.129,0.0712,0.000070,0.1310,0.124000
7qEHsqek33rTcFNT9PFqLf,Someone You Loved,0.501,0.405,109.891,-5.679,0.446,0.0319,0.000000,0.1050,0.751000
12VWzyPDBCc8fqeWCAfNwR,One Dance,0.792,0.625,103.967,-5.609,0.370,0.0536,0.001880,0.3290,0.007760
7crMiinWx373rNBZBaVske,Closer,0.738,0.541,94.949,-5.597,0.657,0.0303,0.000000,0.1100,0.415000
3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,0.760,0.479,89.911,-5.574,0.913,0.0466,0.000000,0.0703,0.556000
6v3KW9xbzN5yKLt9YKDYA2,Señorita,0.759,0.548,116.967,-6.049,0.749,0.0290,0.000000,0.0828,0.039200


In [50]:
nameDf = pd.DataFrame(df["track_name"])
nameDf

,track_name
track_id,
7qiZfU4dY1lWllzX7mPBI3,Shape of You
0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights
1rgnBhdG2JDFTbYkYRZAku,Dance Monkey
0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage)
7qEHsqek33rTcFNT9PFqLf,Someone You Loved
...,...
4EWCNWgDS8707fNSZ1oaA5,Heartless
67iAlVNDDdddxqSD2EZhFs,I'm Gonna Be (500 Miles)
1dGr1c8CrMLDpV6mPbImSI,Lover


In [24]:
playlist_link = "https://open.spotify.com/playlist/2YRe7HRKNRvXdJBp9nXFza?si=ea6c92c1c0c24e03"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [38]:
newDF = pd.DataFrame(sp.playlist_tracks(playlist_URI)["items"])
newDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   added_at         100 non-null    object
 1   added_by         100 non-null    object
 2   is_local         100 non-null    bool  
 3   primary_color    0 non-null      object
 4   track            100 non-null    object
 5   video_thumbnail  100 non-null    object
dtypes: bool(1), object(5)
memory usage: 4.1+ KB


In [33]:
pop_list = []
for track in sp.playlist_tracks(playlist_URI)["items"]: 
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    print(track_name)
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    pop_list.append(track_pop)

Shape of You
Blinding Lights
Dance Monkey
rockstar (feat. 21 Savage)
Someone You Loved
One Dance
Closer
Sunflower - Spider-Man: Into the Spider-Verse
Señorita
Believer
bad guy
Perfect
Say You Won't Let Go
Thinking out Loud
God's Plan
Lucid Dreams
Don't Start Now
Photograph
SAD!
Starboy
Something Just Like This
Shallow
Love Yourself
Bohemian Rhapsody - Remastered 2011
Havana (feat. Young Thug)
lovely (with Khalid)
All of Me
Thunder
7 rings
Take Me to Church
Circles
Jocelyn Flores
goosebumps
XO Tour Llif3
New Rules
Watermelon Sugar
Happier
SICKO MODE
Stressed Out
Lean On (feat. MØ & DJ Snake)
Sorry
HUMBLE.
STAY (with Justin Bieber)
Let Her Go
Better Now
Roses - Imanbek Remix
Let Me Love You
Faded
Don't Let Me Down
Congratulations
Despacito - Remix
Wake Me Up
I Took A Pill In Ibiza - Seeb Remix
Stay With Me
Counting Stars
I Don't Care (with Justin Bieber)
Treat You Better
thank u, next
Without Me
Uptown Funk (feat. Bruno Mars)
Riptide
The Hills
Can't Hold Us (feat. Ray Dalton)
Too Good At

In [40]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [58]:
pop_list = []
track_uri_list = []
artist_name_list = []
artist_pop_list = []
artist_genres_list = []
album_list = []
for track in get_playlist_tracks(username,'2YRe7HRKNRvXdJBp9nXFza'): 
    #URI
    track_uri = track["track"]["uri"]
    track_uri_list.append(track_uri)
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    artist_name_list.append(artist_name)
    artist_pop_list.append(artist_pop)
    artist_genres_list.append(artist_genres)
    
    #Album
    album = track["track"]["album"]["name"]
    album_list.append(album)
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    pop_list.append(track_pop)

In [60]:
nameDf['popularity'] = pop_list
nameDf['track_uri'] = track_uri_list
nameDf['artist_name'] = artist_name_list
nameDf['artist_popularity'] = artist_pop_list
nameDf['artist_genres'] = artist_genres_list
nameDf['album'] = album_list
nameDf.tail(10)

,track_name,popularity,track_uri,artist_name,artist_popularity,artist_genres,album
track_id,,,,,,,
2qxmye6gAegTMjLKEBoR3d,Let Me Down Slowly,88,spotify:track:2qxmye6gAegTMjLKEBoR3d,Alec Benjamin,80,"[alt z, dream smp, electropop, pop]",Narrated For You
5vjLSffimiIP26QG5WcN2K,Hold On,85,spotify:track:5vjLSffimiIP26QG5WcN2K,Chord Overstreet,68,[acoustic pop],Hold On
1HhNoOuqm1a5MXYEgAFl8o,Enemy (with JID) - from the series Arcane Leag...,90,spotify:track:1HhNoOuqm1a5MXYEgAFl8o,Imagine Dragons,92,"[modern rock, rock]",Mercury - Act 1
2TfSHkHiFO4gRztVIkggkE,"Sugar, We're Goin Down",82,spotify:track:2TfSHkHiFO4gRztVIkggkE,Fall Out Boy,82,"[emo, modern rock, rock]",From Under The Cork Tree
7Feaw9WAEREY0DUOSXJLOM,Summer Days (feat. Macklemore & Patrick Stump ...,79,spotify:track:7Feaw9WAEREY0DUOSXJLOM,Martin Garrix,82,"[dance pop, dutch edm, edm, electro house, pop...",Summer Days (feat. Macklemore & Patrick Stump ...
4EWCNWgDS8707fNSZ1oaA5,Heartless,87,spotify:track:4EWCNWgDS8707fNSZ1oaA5,Kanye West,96,"[chicago rap, rap]",808s & Heartbreak
67iAlVNDDdddxqSD2EZhFs,I'm Gonna Be (500 Miles),76,spotify:track:67iAlVNDDdddxqSD2EZhFs,The Proclaimers,65,"[celtic rock, scottish rock]",Sunshine on Leith
1dGr1c8CrMLDpV6mPbImSI,Lover,87,spotify:track:1dGr1c8CrMLDpV6mPbImSI,Taylor Swift,98,[pop],Lover
2iUmqdfGZcHIhS3b9E9EWq,Everybody Talks,85,spotify:track:2iUmqdfGZcHIhS3b9E9EWq,Neon Trees,71,"[modern alternative rock, modern rock, pop roc...",Picture Show


In [66]:
nameDf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 791 entries, 7qiZfU4dY1lWllzX7mPBI3 to 4DpNNXFMMxQEKl7r0ykkWA
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   track_name         791 non-null    object
 1   popularity         791 non-null    int64 
 2   track_uri          791 non-null    object
 3   artist_name        791 non-null    object
 4   artist_popularity  791 non-null    int64 
 5   artist_genres      791 non-null    object
 6   album              791 non-null    object
dtypes: int64(2), object(5)
memory usage: 49.4+ KB


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 791 entries, 7qiZfU4dY1lWllzX7mPBI3 to 4DpNNXFMMxQEKl7r0ykkWA
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_name        791 non-null    object 
 1   danceability      791 non-null    float64
 2   energy            791 non-null    float64
 3   tempo             791 non-null    float64
 4   loudness          791 non-null    float64
 5   valence           791 non-null    float64
 6   speechiness       791 non-null    float64
 7   instrumentalness  791 non-null    float64
 8   liveness          791 non-null    float64
 9   acousticness      791 non-null    float64
dtypes: float64(9), object(1)
memory usage: 68.0+ KB


In [84]:
finalDF = nameDf.merge(df, how='inner', on='track_id')
finalDF

,track_name_x,popularity,track_uri,artist_name,artist_popularity,artist_genres,album,track_name_y,danceability,energy,tempo,loudness,valence,speechiness,instrumentalness,liveness,acousticness
track_id,,,,,,,,,,,,,,,,,
7qiZfU4dY1lWllzX7mPBI3,Shape of You,89,spotify:track:7qiZfU4dY1lWllzX7mPBI3,Ed Sheeran,96,"[pop, uk pop]",÷ (Deluxe),Shape of You,0.825,0.652,95.977,-3.183,0.931,0.0802,0.000000,0.0931,0.58100
0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,95,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,The Weeknd,97,"[canadian contemporary r&b, canadian pop, pop]",After Hours,Blinding Lights,0.514,0.730,171.005,-5.934,0.334,0.0598,0.000095,0.0897,0.00146
1rgnBhdG2JDFTbYkYRZAku,Dance Monkey,68,spotify:track:1rgnBhdG2JDFTbYkYRZAku,Tones And I,78,"[australian pop, pop]",Dance Monkey,Dance Monkey,0.826,0.593,98.083,-6.401,0.541,0.0976,0.000161,0.1700,0.68800
0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage),88,spotify:track:0e7ipj03S05BNilyu5bRzt,Post Malone,91,"[dfw rap, melodic rap, rap]",beerbongs & bentleys,rockstar (feat. 21 Savage),0.585,0.520,159.801,-6.136,0.129,0.0712,0.000070,0.1310,0.12400
7qEHsqek33rTcFNT9PFqLf,Someone You Loved,90,spotify:track:7qEHsqek33rTcFNT9PFqLf,Lewis Capaldi,82,"[pop, uk pop]",Divinely Uninspired To A Hellish Extent,Someone You Loved,0.501,0.405,109.891,-5.679,0.446,0.0319,0.000000,0.1050,0.75100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4EWCNWgDS8707fNSZ1oaA5,Heartless,87,spotify:track:4EWCNWgDS8707fNSZ1oaA5,Kanye West,96,"[chicago rap, rap]",808s & Heartbreak,Heartless,0.790,0.647,87.999,-5.983,0.654,0.1360,0.000000,0.2480,0.05150
67iAlVNDDdddxqSD2EZhFs,I'm Gonna Be (500 Miles),76,spotify:track:67iAlVNDDdddxqSD2EZhFs,The Proclaimers,65,"[celtic rock, scottish rock]",Sunshine on Leith,I'm Gonna Be (500 Miles),0.851,0.551,131.922,-5.177,0.807,0.0396,0.000000,0.0827,0.15100
1dGr1c8CrMLDpV6mPbImSI,Lover,87,spotify:track:1dGr1c8CrMLDpV6mPbImSI,Taylor Swift,98,[pop],Lover,Lover,0.359,0.543,68.534,-7.582,0.453,0.0919,0.000016,0.1180,0.49200


In [92]:
finalDF.iloc[546]

track_name_x                                     Bitter Sweet Symphony
popularity                                                          83
track_uri                         spotify:track:57iDDD9N9tTWe75x6qhStw
artist_name                                                  The Verve
artist_popularity                                                   69
artist_genres        [alternative rock, britpop, permanent wave, po...
album                                    Urban Hymns (Remastered 2016)
track_name_y                                     Bitter Sweet Symphony
danceability                                                     0.356
energy                                                           0.917
tempo                                                          171.176
loudness                                                        -6.006
valence                                                          0.518
speechiness                                                     0.0479
instru

In [93]:
finalDF.to_csv('SpotifyFinalAPI.csv')